In [39]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import harmonypy
import pymn
import scrublet as scr
import gseapy as gp
from gseapy.plot import barplot, dotplot
import bottleneck
import pyreadr
from scipy.io import mmread
import scipy
from sklearn import preprocessing
import random as rd
import scib
from matplotlib_venn import venn2,venn2_circles,venn3
from pyscenic.export import export2loom, add_scenic_metadata
from sklearn.decomposition import NMF
from sklearn import metrics
import palettable
from pyscenic.rss import regulon_specificity_scores
import matplotlib.image as mpimg
import cosg as cosg
from pySankey.sankey import sankey

import matplotlib as mpl

Error: package or namespace load failed for 'methods' in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/public/home/guogjgroup/ggj/anaconda3/envs/R3.6/lib/R/library/methods/libs/methods.so':
  libR.so: cannot open shared object file: No such file or directory
During startup - Warning messages:
1: package 'methods' was built under R version 3.6.3 
2: package "methods" in options("defaultPackages") was not found 
3: package 'datasets' was built under R version 3.6.3 
4: package 'grDevices' was built under R version 3.6.3 
5: package 'graphics' was built under R version 3.6.3 
6: package 'methods' was built under R version 3.6.3 
7: Setting LC_CTYPE failed, using "C" 


In [40]:
sc.set_figure_params(dpi = 200, color_map = 'viridis_r' )
sc.settings.verbosity = 2

/public/home/guogjgroup/ggj/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [ ]:
# https://github.com/aertslab/SCENIC/blob/master/vignettes/SCENIC_Running.Rmd
# *Subsampling cells*: When there is a high proportion of low-quality cells, or if the computation time is an issue, it is also possible to infer the regulatory network using a subset of cells (e.g. selecting random or high-quality cells as input to the co-expression analysis). The activity of the regulatory network, *trained* on this subset of cells, can then be evaluated on all the cells in the dataset with AUCell (Step 3).
# Note that to avoid loss of resolution, the subset of cells should be representative of the whole dataset (e.g. contain sufficient representation of all the cell types).
# Examples of this approach are presented in [*@aibar2017*](http://dx.doi.org/10.1038/nmeth.4463) (i.e. subsampling this mouse brain dataset, and the analysis of 49k cells from mouse retina).


# Subsampling for SCENIC: Epithelium

In [23]:
# High

adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cancer_module/cnv_high_epi_harmony/cnv_high_epi_harmony_anno_latest.h5ad')

barcode_list_all = []

for each_cancer in np.unique(adata.obs['cancer_type']).tolist():
    
    adata_subset = adata[adata.obs['cancer_type'] == each_cancer].copy()
    
    for each_cluster in np.unique(adata_subset.obs['leiden']).tolist():
        
        if adata_subset[adata_subset.obs['leiden'] == each_cluster].shape[0] > 1000:
        
            barcode_list = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/scripts_share/jikai/hhseq_bam_integration_split/bam_subset/' + each_cancer + '_high_' + str(each_cluster) + '_bam_subset.csv', index_col=0, header=None).index.tolist()
        
        else: 
        
            barcode_list = adata_subset[adata_subset.obs['leiden'] == each_cluster].obs.index.tolist()
            
        barcode_list_all.extend(barcode_list)
        
barcode_list_all = np.unique(barcode_list_all).tolist()

adata = adata[barcode_list_all, :]

adata_high = adata.copy()

# Low

adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cnv_high_DEG/cnv_high_differential_gene_expression_raw_all_gene.h5ad')

adata = adata[adata.obs['cnv_cluster_type'] == 'low']

barcode_list_all = []

for each_patient in np.unique(adata.obs['study_id']).tolist():
        
    if adata[adata.obs['study_id'] == each_patient].shape[0] > 1000:
    
        barcode_list = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/scripts_share/jikai/hhseq_bam_integration_split/bam_subset/' + each_patient + '_low_bam_subset.csv', index_col=0, header=None).index.tolist()
            
    else: 
        
        barcode_list = adata[adata.obs['study_id'] == each_patient].obs.index.tolist()
        
    barcode_list_all.extend(barcode_list)

barcode_list_all = np.unique(barcode_list_all).tolist()

adata = adata[barcode_list_all, :]

adata_low = adata.copy()

del adata

adata = sc.AnnData.concatenate(adata_low, adata_high, join = "outer", fill_value = 0, index_unique = None)

del adata_high
del adata_low

sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
sc.pp.filter_genes(adata, min_cells=20)

latest_anno = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/hhseq_all_cell_anno_latest.csv', index_col=0)
latest_anno = latest_anno.loc[adata.obs.index.tolist(), :]

adata.obs['Type'] = latest_anno['Type']
adata.obs['Celltype'] = latest_anno['Celltype']
adata.obs['lineage'] = latest_anno['lineage']

adata.write('SCENIC_Epithelium_subset_1/SCENIC_Epithelium_subset_1.h5ad')

/public/home/guogjgroup/ggj/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


filtered out 5579 genes that are detected in less than 20 cells


/tmp/ipykernel_31033/1447749131.py:65: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  latest_anno = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/hhseq_all_cell_anno_latest.csv', index_col=0)


In [30]:
# High

adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cancer_module/cnv_high_epi_harmony/cnv_high_epi_harmony_anno_latest.h5ad')

barcode_list_all = []

for each_cancer in np.unique(adata.obs['cancer_type']).tolist():
    
    adata_subset = adata[adata.obs['cancer_type'] == each_cancer].copy()
    
    for each_cluster in np.unique(adata_subset.obs['leiden']).tolist():
        
        if adata_subset[adata_subset.obs['leiden'] == each_cluster].shape[0] > 1000:
        
            patient_subset = adata_subset[adata_subset.obs['leiden'] == each_cluster].copy()
            
            barcode_list = sc.pp.subsample(patient_subset, n_obs=1000, copy=True, random_state=10).obs.index.tolist()
         
        else: 
        
            barcode_list = adata_subset[adata_subset.obs['leiden'] == each_cluster].obs.index.tolist()
            
        barcode_list_all.extend(barcode_list)
        
barcode_list_all = np.unique(barcode_list_all).tolist()

adata = adata[barcode_list_all, :]

adata_high = adata.copy()

# Low

adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cnv_high_DEG/cnv_high_differential_gene_expression_raw_all_gene.h5ad')

adata = adata[adata.obs['cnv_cluster_type'] == 'low']

barcode_list_all = []

for each_patient in np.unique(adata.obs['study_id']).tolist():
        
    if adata[adata.obs['study_id'] == each_patient].shape[0] > 1000:
    
        patient_subset = adata[adata.obs['study_id'] == each_patient].copy()

        barcode_list = sc.pp.subsample(patient_subset, n_obs=1000, copy=True, random_state=10).obs.index.tolist()
             
    else: 
        
        barcode_list = adata[adata.obs['study_id'] == each_patient].obs.index.tolist()
        
    barcode_list_all.extend(barcode_list)

barcode_list_all = np.unique(barcode_list_all).tolist()

adata = adata[barcode_list_all, :]

adata_low = adata.copy()

del adata

adata = sc.AnnData.concatenate(adata_low, adata_high, join = "outer", fill_value = 0, index_unique = None)

del adata_high
del adata_low

sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
sc.pp.filter_genes(adata, min_cells=20)

latest_anno = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/hhseq_all_cell_anno_latest.csv', index_col=0)
latest_anno = latest_anno.loc[adata.obs.index.tolist(), :]

adata.obs['Type'] = latest_anno['Type']
adata.obs['Celltype'] = latest_anno['Celltype']
adata.obs['lineage'] = latest_anno['lineage']

adata.write('SCENIC_Epithelium_subset_2/SCENIC_Epithelium_subset_2.h5ad')

/public/home/guogjgroup/ggj/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


filtered out 5550 genes that are detected in less than 20 cells


/tmp/ipykernel_31033/3257897941.py:69: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  latest_anno = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/hhseq_all_cell_anno_latest.csv', index_col=0)


In [24]:
# High

adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cancer_module/cnv_high_epi_harmony/cnv_high_epi_harmony_anno_latest.h5ad')

barcode_list_all = []

for each_cancer in np.unique(adata.obs['cancer_type']).tolist():
    
    adata_subset = adata[adata.obs['cancer_type'] == each_cancer].copy()
    
    for each_cluster in np.unique(adata_subset.obs['leiden']).tolist():
        
        if adata_subset[adata_subset.obs['leiden'] == each_cluster].shape[0] > 1000:
        
            barcode_list = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/scripts_share/jikai/hhseq_bam_integration_split/bam_subset/' + each_cancer + '_high_' + str(each_cluster) + '_bam_subset_3.csv', index_col=0, header=None).index.tolist()
        
        else: 
        
            barcode_list = adata_subset[adata_subset.obs['leiden'] == each_cluster].obs.index.tolist()
            
        barcode_list_all.extend(barcode_list)
        
barcode_list_all = np.unique(barcode_list_all).tolist()

adata = adata[barcode_list_all, :]

adata_high = adata.copy()

# Low

adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cnv_high_DEG/cnv_high_differential_gene_expression_raw_all_gene.h5ad')

adata = adata[adata.obs['cnv_cluster_type'] == 'low']

barcode_list_all = []

for each_patient in np.unique(adata.obs['study_id']).tolist():
        
    if adata[adata.obs['study_id'] == each_patient].shape[0] > 1000:
    
        barcode_list = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/scripts_share/jikai/hhseq_bam_integration_split/bam_subset/' + each_patient + '_low_bam_subset_3.csv', index_col=0, header=None).index.tolist()
            
    else: 
        
        barcode_list = adata[adata.obs['study_id'] == each_patient].obs.index.tolist()
        
    barcode_list_all.extend(barcode_list)

barcode_list_all = np.unique(barcode_list_all).tolist()

adata = adata[barcode_list_all, :]

adata_low = adata.copy()

del adata

adata = sc.AnnData.concatenate(adata_low, adata_high, join = "outer", fill_value = 0, index_unique = None)

del adata_high
del adata_low

sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
sc.pp.filter_genes(adata, min_cells=20)

latest_anno = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/hhseq_all_cell_anno_latest.csv', index_col=0)
latest_anno = latest_anno.loc[adata.obs.index.tolist(), :]

adata.obs['Type'] = latest_anno['Type']
adata.obs['Celltype'] = latest_anno['Celltype']
adata.obs['lineage'] = latest_anno['lineage']

adata.write('SCENIC_Epithelium_subset_3/SCENIC_Epithelium_subset_3.h5ad')

/public/home/guogjgroup/ggj/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


filtered out 5540 genes that are detected in less than 20 cells


/tmp/ipykernel_31033/628378602.py:65: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  latest_anno = pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/hhseq_all_cell_anno_latest.csv', index_col=0)
